In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import io
from math import sin, cos, sqrt, atan2, radians
from math import radians, cos, sin, asin, sqrt
import matplotlib.pyplot as plt
from csv import reader
import sys
from bs4 import BeautifulSoup
from flatten_json import flatten
import json
from thefuzz import fuzz
from fuzzywuzzy import fuzz
from deep_translator import GoogleTranslator

c:\Users\123\AppData\Local\Programs\Python\Python311\Lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
c:\Users\123\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
URL_AZ = pd.DataFrame()
URL_AZ = []
URL_CP_ALL = []
URL_CP = []
URL_DT = []
URL_TEST = []
Product_name=[]
Compound_name=[]
_url = 'https://www.medicines.org.uk/emc/browse-ingredients'
df=pd.DataFrame()
Indication=[]
Contrain=[]
Special_warnings=[]
Side_Effect=[]
gt = GoogleTranslator(source='en', target='vi')

In [4]:
#Lây url từ a đến z , sau đó thêm vào list URL_AZ
soup = BeautifulSoup(requests.get(_url).text, 'html.parser')
_a = soup.find_all(attrs={"class": "emc-link"})
for a in _a:
    ref =  a.get('href','')
    if '/browse-ingredients/' in ref:
        url_ = 'https://www.medicines.org.uk' + ref
        URL_AZ.append(url_)
print(len(URL_AZ))


27


In [248]:
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

In [249]:
# Lấy ví dụ  1 trang đầu tiên là Page A
# for i in range(1,len(URL_AZ)):
soup_1 = BeautifulSoup(requests.get(URL_AZ[1]).text, 'html.parser')
page_a = soup_1.find_all(attrs={"class": "emc-link"})
for a in page_a:
    ref_1 =  a.get('href','')
    if '/ingredient/' in ref_1:
        url_1 = 'https://www.medicines.org.uk' + ref_1
        URL_CP_ALL.append(url_1)
print(len(URL_CP_ALL))
    

171


In [250]:
# Lấy mẫu dược chất đầu tiên trong list URL_CP_ALL
for i in range(len(URL_CP_ALL)):
    soup_2 = BeautifulSoup(requests.get(URL_CP_ALL[i]).text, 'html.parser')
    _c = soup_2.find_all(attrs={"class": "search-results-product-info-title-link emc-link"})
    _detail = soup_2.find_all(attrs={"class": "search-results-product-info-type emc-text-default"})
    for c in _c:
        ref_2 = c.get('href','')
        if '/emc/product/' in ref_2:
            url_2 = 'https://www.medicines.org.uk' + ref_2
            URL_DT.append(url_2)
    for c in _c:
        Product_name.append(cleanhtml(str(c)).strip())
    for d in _detail:
        Compound_name.append(cleanhtml(str(d)).strip())
print('Len của URL_DT',len(URL_DT))
print('Len của Product_Name',len(Product_name))
print('Len của Compound_Name',len(Compound_name))

Len của URL_DT 905
Len của Product_Name 905
Len của Compound_Name 905


In [251]:

for i in range(len(URL_DT)):
    soup_3 = BeautifulSoup(requests.get(URL_DT[i]).text, 'html.parser')
    elements = soup_3.find_all(class_='spcWrapper')
    div_wrapper = soup_3.find('div', class_='spcWrapper')
    
    indication_41=soup_3.find(id='INDICATIONS')
    side_effect_48=soup_3.find(id='UNDESIRABLE_EFFECTS')
    contrai_43=soup_3.find(id='CONTRAINDICATIONS')
    contrai_44=soup_3.find(id='CLINICAL_PRECAUTIONS')
    if indication_41 is not None or side_effect_48 is not None or contrai_43 is not None or contrai_44 is not None:
        contrain_Wrapper_43 = contrai_43.find_next_siblings(class_="sectionWrapper")
        contrain_Wrapper_44 = side_effect_48.find_next_siblings(class_="sectionWrapper")
        indication_Wrapper_41 = indication_41.find_next_siblings(class_="sectionWrapper")
        side_effect_Wrapper_48 = contrai_44.find_next_siblings(class_="sectionWrapper")


        wrapper_43=cleanhtml(str(contrain_Wrapper_43))
        wrapper_44=cleanhtml(str(contrain_Wrapper_44))
        wrapper_41=cleanhtml(str(indication_Wrapper_41))
        wrapper_48=cleanhtml(str(side_effect_Wrapper_48))


        wrapper_43_ = wrapper_43.replace("[", "").replace("]", "").strip()
        wrapper_44_ = wrapper_44.replace("[", "").replace("]", "").strip()
        wrapper_41_ = wrapper_41.replace("[", "").replace("]", "").strip()
        wrapper_48_ = wrapper_48.replace("[", "").replace("]", "").strip()


        Indication.append(wrapper_41_)
        Contrain.append(wrapper_43_)
        Special_warnings.append(wrapper_44_)
        Side_Effect.append(wrapper_48_)
    else:
        pass
    

print('Len của Indication',len(Indication))
print('Len của Contrain',len(Contrain))
print('Len của Special_warnings',len(Special_warnings))
print('Len của Side_Effect',len(Side_Effect))



Len của Indication 835
Len của Contrain 835
Len của Special_warnings 835
Len của Side_Effect 835


In [ ]:
df['Compound_Name']=Compound_name
df['Product_Name']=Product_name
df['Indication']=Indication
df['Contraindication']=Contrain
df['Special_warnings']=Special_warnings
df['Side_Effect']=Side_Effect

In [162]:
df

,Compound_Name,Product_Name,Indication,Contraindication,Special_warnings,Side_Effect
0,abacavir,Abacavir 300 mg Film-Coated Tablets,Abacavir is indicated in antiretroviral combin...,Hypersensitivity to abacavir or to any of the ...,"For many adverse reactions reported, it is unc...",Hypersensitivity reactions (see also section 4...
1,abacavir,Abacavir Mylan 300 mg Film-coated Tablets,Abacavir Mylan is indicated in antiretroviral ...,Hypersensitivity to the active substance or to...,"For many adverse reactions reported, it is unc...",Hypersensitivity reactions (see also section 4...
2,"lamivudine, abacavir",Abacavir/Lamivudine 600 mg/300 mg film-coated ...,Abacavir/Lamivudine is indicated in antiretrov...,Hypersensitivity to the active substances or t...,Summary of the safety profile\n\nThe adverse r...,The special warnings and precautions relevant ...
3,"lamivudine, abacavir",Abacavir/Lamivudine Dr. Reddys 600 mg/300 mg F...,Abacavir/Lamivudine is indicated in antiretrov...,Hypersensitivity to the active substances or t...,Summary of the safety profile\n\nThe adverse r...,The special warnings and precautions relevant ...


In [166]:
df.to_excel('/FRT/19.Contraindication/5.Crawl_data_micidines/1.Data_Crawl/check_2.xlsx')

In [164]:
df

,Compound_Name,Product_Name,Indication,Contraindication,Special_warnings,Side_Effect
0,abacavir,Abacavir 300 mg Film-Coated Tablets,Abacavir is indicated in antiretroviral combin...,Hypersensitivity to abacavir or to any of the ...,"For many adverse reactions reported, it is unc...",Hypersensitivity reactions (see also section 4...
1,abacavir,Abacavir Mylan 300 mg Film-coated Tablets,Abacavir Mylan is indicated in antiretroviral ...,Hypersensitivity to the active substance or to...,"For many adverse reactions reported, it is unc...",Hypersensitivity reactions (see also section 4...
2,"lamivudine, abacavir",Abacavir/Lamivudine 600 mg/300 mg film-coated ...,Abacavir/Lamivudine is indicated in antiretrov...,Hypersensitivity to the active substances or t...,Summary of the safety profile\n\nThe adverse r...,The special warnings and precautions relevant ...
3,"lamivudine, abacavir",Abacavir/Lamivudine Dr. Reddys 600 mg/300 mg F...,Abacavir/Lamivudine is indicated in antiretrov...,Hypersensitivity to the active substances or t...,Summary of the safety profile\n\nThe adverse r...,The special warnings and precautions relevant ...
